### Extract the coding sequence portion of all reads spanning a given gene as well as the associated phred scores
---

Workflow:
- genome .gff
- gene .ini
    - Use these to produce a gene specific .bed file
- sample .bam file
    - Use this + .bed file to produce gene specific .sam file 
        - only contains reads overlapping entire coding sequence
        
Questions:
- Does it handle multiple exons?
- How to handle deletions in the phred array?
    - Are their quality scores?

In [1]:
import os
import sys
import configparser
import getopt
import numpy as np
import pandas as pd
from collections import Counter

from lib.mutation import *
from lib.error import *
from lib.coi import *

In [2]:
from matplotlib import patches
from matplotlib import lines
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec

# inline stylization
%matplotlib inline
sns.set_style('white')
sns.set_style('ticks')
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['figure.figsize'] = [4, 4]

## 0 Settings

In [8]:
platform = "ont"
country = "zambia"
date = "2019-04-10"
sample = "BC05"
gene = "msp2"

In [9]:
analysis_dir = os.path.join("../analysis", platform, country, date)
results_dir = os.path.join("../results", platform, country, date)
output_dir = os.path.join(analysis_dir, "coi")

In [10]:
gene_ini = os.path.join("../data/resources/pf-regions/", gene + ".ini")
gene_ini_path = os.path.dirname(gene_ini)

In [11]:
config = configparser.ConfigParser()
config.read(gene_ini)
gene_dt = {}
gene_dt["name"] = config.get("Parameters", "name")
gene_dt["id"] = config.get("Parameters", "id")
gene_dt["genome"] = config.get("Parameters", "genome")
gene_dt["gff"] = config.get("Parameters", "gff")

In [12]:
os.listdir(output_dir)

['BC04.MSP2.gene_array.npy',
 'BC05.MSP2.gene_array.npy',
 'BC06.MSP2.gene_array.npy',
 'BC06.MSP2.phred_array.npy',
 'BC01.MSP2.gene_array.npy',
 'BC01.MSP2.sam',
 'BC03.MSP2.gene_array.npy',
 'BC02.MSP2.gene_array.npy',
 'BC06.MSP2.sam',
 'BC03.MSP2.sam',
 'BC02.MSP2.sam',
 'BC04.MSP2.sam',
 'BC05.MSP2.sam']

## 1  Produce a `.bed` file delimiting gene exon boundaries

In [13]:
exon_gff = "%s.exons.gff" % os.path.join(gene_ini_path, gene_dt["name"])
exon_bed = "%s.exons.bed" % os.path.join(gene_ini_path, gene_dt["name"])

In [14]:
if not os.path.isfile(exon_gff) or not os.path.isfile(exon_bed):
    print("Creating .gff and .bed file for %s" % gene_dt["name"])
    # Create a GFF describing the exons of the target gene
    cmd = "grep -E 'exon.*%s' %s > %s" % (gene_dt["id"],
                                          gene_dt["gff"],
                                          exon_gff)
    os.system(cmd)
    exon_df = pd.read_csv(exon_gff, sep="\t", header=-1)
    gff_columns = ["seq", "source", "feature", 
                   "start", "end", "score", "strand", 
                   "phase", "attributes"]
    exon_df.columns = gff_columns
    exon_df.start = exon_df.start - 1  # Off by one incongruity with mpileup, unfortunately.
    exon_df.to_csv(exon_gff, sep="\t", index=False, header=False)

    # Create an associated BED file
    cmd = "cut -f 1,4,5 %s > %s" % (exon_gff, exon_bed)
    os.system(cmd)
else:
    print(".gff and .bed file for %s already exist." % gene_dt["name"])
    exon_df = pd.read_csv(exon_gff, sep="\t", header=-1)
    gff_columns = ["seq", "source", "feature", 
                   "start", "end", "score", "strand", 
                   "phase", "attributes"]
    exon_df.columns = gff_columns

.gff and .bed file for MSP2 already exist.


## 2 Produce a sample-gene-specific `.sam`

In [20]:
gene_sam_fn = "%s.%s.sam" % (sample, gene_dt["name"])
sam_path = os.path.join(output_dir, gene_sam_fn)
print("Output path for .sam:", sam_path)

Output path for .sam: ../analysis/ont/zambia/2019-04-10/coi/BC05.MSP2.sam


In [21]:
if not os.path.isfile(sam_path):
    cmd = "samtools view %s -L %s > %s" % (bam_path,
                                           exon_bed,
                                           sam_path)
    print("Generating a .sam file containing %s overlapping reads ONLY." % gene_dt["name"])
    print("  Command:", cmd)
    os.system(cmd)
else:
    print(".sam file already exists.")

.sam file already exists.


## 3 Extract `gene_array` and `phred_array`

In [22]:
verbose = False

In [25]:
def convert_qual(qual, shift_int=33):
    """
    Convert an ASCII encoded quality score into
    a Phred score or probability of error
    
    """
    return np.array([ord(c) for c in qual]) - shift_int

In [26]:
with open(sam_path, "r+") as sam:
    gene_start = exon_df.start[0]
    gene_end = exon_df.end[0]
    gene_length = gene_end - gene_start
    
    if verbose:
        print("Gene start:", gene_start)
        print("Gene end:", gene_end)
        print("Gene length:", gene_length)
    
    gene_phreds = []
    gene_ints = []
    flags = []
    for line in sam:
        _, flag, _, start, _, cigar, _, _, _, seq, qual = line.split()[:11]
                
        if verbose:
            print("Start:", start)
            print("CIGAR:", cigar[:10], " ...")
            print("Sequence:", seq[:10], " ...")
            print("Quality:", qual[:10], "...")
        
        
        # Process the CIGAR string of each read,
        # assigning reference position to each base
        start = int(start)
        positions, mapped_seq = process_by_cigar(start, seq, cigar)
        
        # Filter for bases within coding region
        gene_bases = [c 
                      for c, p in zip(mapped_seq, positions) 
                      if gene_start < p <= gene_end]
        
        # Reverse complement if necessary
        if exon_df["strand"][0] == '-':
            gene_bases = [complement_map[c] for c in gene_bases[::-1]]
        
        # If the read spanned the entire coding region, save
        if len(gene_bases) == gene_length:
            gene_int = np.array([dna_to_ints[c] for c in gene_bases])
            gene_ints.append(gene_int)
            flags.append(flag)
            
        # Repeat about but for quality scores
        positions, mapped_qual = process_by_cigar(start, qual, cigar) 
        gene_qual = [c
                     for c, p in zip(mapped_qual, positions)
                     if gene_start < p <= gene_end]
        
        if exon_df["strand"][0] == '-':
            gene_qual = [c for c in gene_qual[::-1]]  # simply reverse order
        
        # If the read spanned the entire coding region, save
        if len(gene_qual) == gene_length:
            gene_phred = convert_qual(gene_qual)
            gene_phred[gene_int == -1] = -1  # these are where deletions occured
            gene_phreds.append(gene_phred)

In [27]:
gene_array = np.vstack(gene_ints)
phred_array = np.vstack(gene_phreds)

In [28]:
print("Gene array:", gene_array.shape)
print("PHRED array:", phred_array.shape)

Gene array: (43228, 819)
PHRED array: (43228, 819)


## 4 Write

In [33]:
gene_fn = "%s.%s.gene_array.npy" % (sample, gene_dt["name"])
phred_fn = "%s.%s.phred_array.npy" % (sample, gene_dt["name"])

In [34]:
print("Gene file:", gene_fn)
print("PHRED file:", phred_fn)

Gene file: BC05.MSP2.gene_array.npy
PHRED file: BC05.MSP2.phred_array.npy


In [35]:
np.save(os.path.join(output_dir, gene_fn), gene_array)
np.save(os.path.join(output_dir, phred_fn), phred_array)

In [36]:
os.listdir(output_dir)

['BC04.MSP2.gene_array.npy',
 'BC05.MSP2.gene_array.npy',
 'BC06.MSP2.gene_array.npy',
 'BC06.MSP2.phred_array.npy',
 'BC01.MSP2.gene_array.npy',
 'BC01.MSP2.sam',
 'BC03.MSP2.gene_array.npy',
 'BC02.MSP2.gene_array.npy',
 'BC06.MSP2.sam',
 'BC03.MSP2.sam',
 'BC02.MSP2.sam',
 'BC05.MSP2.phred_array.npy',
 'BC04.MSP2.sam',
 'BC05.MSP2.sam']